# TDD in data analysis
## A step-by-step tutorial

The previous two posts of this series were about the [business](https://medium.com/@torokagoston/effective-data-science-teams-part-1-9f40c6bff275) and [people](https://medium.com/@torokagoston/effective-data-science-teams-part-2-6af6f7674be9) perspectives of effective data science teams. However without the right tools and processes the recipe does not work. Therefore in the next parts I'm going to discuss some of the tools and processes that I consider important for a data science team. This part is going to be about how to consolidate the practices of data analysis with the practices of software development. The specific process that I'm going to focus on is test-driven development for data analysis. At first I intended this also as a post around stories and conclusions like the previous ones. But since I did not find a good tutorial from scratch to a working model online, I've decided to make one in the scope of this post. So in the following we discuss the reason of TDD for data analysis and then I present a complete step-by-step tutorial on an example project from Kaggle using TDD.

## Why TDD for data analytics

Test-driven development is a popularized by Kent Beck, the father of extreme programming. It helps the development to focus on the required features and helps rapid iteration of versions. The idea is that when you have a new feature that you have to add to the code first add a test that expects that feature to be present and see it FAIL. Then write the quickest solution to PASS the test. Once it was passed take some time to REFACTOR the code. All three parts of the process are equally important and they work best in this order. Take the first step, fail. I remember when some time ago, we were sitting in front of the computer with my colleagues and were looking at why changing the input parameters of a function does not change the output in the expected way although there was a unittest covering exactly that function. After some investigation it turned out that the mentioned unittest I wrote _after_ the function was written actually never failed. The other important thing is to focus on only passing the actual test and not writing code that is intended to pass all possible tests in the future. One of the guidelines behind is 'Keep it simple, stupid' (KISS). Also the refactoring should not mean changing the test logic or completely rewriting the function that passed it, refactor only as much as absolutely needed. Don't forget the code is your enemy.

Now one can say: alright this is nice and true for writing software for production but how does it relate to my practice when I do exploratory data science or when I just try out different things. First, I'm sure you met the situation when something that you calculated in your notebook on jupyter is not working in the next stage of the development or even when you rerun it a second time. Maybe the fields were not matching, maybe the datatypes, or maybe a seed was not fixed. Second, it should sound familiar when the data just drags you down into the rabbithole: you notice that a linear model does not explain enough variance so you try some non-linear ones, then you see also that some missing values can be predicted so you predict those, then you see that maybe the data should be transformed, some string columns should be vectorized other should be levels of an ordinal scale etc. Then 2 weeks into the exploration the manager asks where is the result that you promised for one week ago and you can only explain him what will be your next steps... and you still did not reduce the uncertainty for her and she still does not know if the feature/model is needed at all or we should concentrate on other promising directions. So I think there are serious reasons to pracitce TDD for data analysis. On top of these empirical considerations, there are theoretical ones as well. TDD and scientific methodology. When you create a test and see it failing is basically making a scientific hypothesis that is [falsifiable](https://en.wikipedia.org/wiki/Falsifiability). When you pass the test that is basically equivalent to the statistical testing of your experimental manipulation in a research study. Finally, refactoring is the similar to the funciton of the discussion part of a paper, you turn your experimental manipulation to a theory. 

## Testing Sanity and Insanity

Although software testing has its own jargon, I considered for some time that instead of unittests or integration tests to think about __insanity__ and __sanity__ tests when it comes to data analysis. Testing insanity means that you want to make sure that the function that you wrote is not producing non-sensical results, it gives some result for input A and some other result for input B etc. i.e. it is not insane. However, no client is going to pay for a solution that is guaranteed to be not insane (100% test coverage). This is only a necessary but not the sufficient condition. For that our solution has to be sane. Therefore sanity testing tests some more higher level and even abstract hypothesis, e.g. that your solution (e.g. a neural network) is better than a simple arithmetic mean or that you are not overfitting the training data. These are features which can be translated easily into direct business value. A financial software requires very strong arguments to not stick to an interpretable model and go for a deep learning solution when they can lose money. Also when you are already overfitting the training data when you compare it to the validation sets then imagine the how much difference you are going to see between the expected fit and real for a dataset that was collected in a later batch that you have not seen during design. The way I think about these tests passing sanity tests make software that you sell and passing insanity tests warrants customer satisfaction in the upcoming months and years of using your product. 

Nevertheless as the saying goes, it is often difficult to tell the fine line between a madman and a genius. The same goes for the question of 'should I put this test in the insanity or in the insanity bucket?'. So, at this point it is time to delve into the step-by-step tutorial. I took an example dataset from Kaggle, the [House Prices dataet](https://www.kaggle.com/c/house-prices-advanced-regression-techniques) this is a sufficiently easy and fun data. Just right the pass the imaginary test of 'tutorial on TDD for analysis'. 

## Step-by-step TDD in a data science task

Since it was a csv file, I started by reading the data with Pandas. The first thing I wanted to check is if there are NaN/NULL values.

In [1]:
%pylab inline
import pandas as pd

train = pd.read_csv('train.csv', index_col=['Id'])
train.head()

Populating the interactive namespace from numpy and matplotlib


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [2]:
sum_nulls = train.isnull().sum()
datatypes = train.dtypes

summary = pd.DataFrame(dict(SumOfNulls = sum_nulls, DataTypes = datatypes))
summary.sort_values(by='SumOfNulls', ascending=False)

,SumOfNulls,DataTypes
PoolQC,1453,object
MiscFeature,1406,object
Alley,1369,object
Fence,1179,object
FireplaceQu,690,object
LotFrontage,259,float64
GarageYrBlt,81,float64
GarageCond,81,object
GarageType,81,object
GarageFinish,81,object


Yep, there are. Usually missing data is inherent to data. If you don't find any in your data then start to be suspicious. Nevertheless, it is good to start the test-driven part to come up with a test that checks if there are NaN values in the data before we go on analysing it. It's easy also, since missing data will not make sense for several model types it's an insanity test. Usually, I first write the test also quick and dirty in the notebook.

In [3]:
def test_no_nan_values(data):
    assert not data.isnull().any(axis=None)
    
test_no_nan_values(train)

AssertionError: 

Good, it failed. Also note that neither I have docstrings at this moment nor this is a proper unittest/pytest case. It's a test. And it fails. That was the purpose of the first stage, so let's move on to the next. 

Missing values are tricky. Pandas has is schemaless which makes prototyping easy but does not help in this case. E.g. missing values are filled with np.nan in otherwise string fields. Nevertheless, this is an intentional feature of pandas which makes easier to use some generic dataframe functions. In my case I don't really want this to happen though. Let's see what we can do with NaN values:

   1. Add 'Unknown' as a separate string to replace NaNs so we can use these fields later - What should be the limit of NaN values where we apply this strategy, obviously you don't want to spend too much energy on columns which have 95% missing values in the first iteration. So 90? 75? This should be tested..

   2. Predict the missing values. - This would take lot's of models and maybe different strategies for different fields.

   3. Ommit columns with missing values. - Since I want to pass the test first with the least effort (and from the higher level perspective bring results the soonest possible) I will choose this. 

I looked at the test set and because there even more NaN values were present I decided to take the union of columns with missing values in the two dataset. Also Since I want to do the same operations on the two sets I decided to quickly create a common class for them

In [4]:
class HousePriceData(object):
    def __init__(self, filename):
        self.X_cols = ['MSSubClass', 'LotArea', 'Street', 'LotShape', 'LandContour',
                       'LotConfig', 'LandSlope', 'Condition1', 'Condition2',
                       'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt',
                       'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond',
                       'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 
                       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath',
                       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
                       'Fireplaces', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
                       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
                       'MoSold', 'YrSold', 'SaleCondition']
        self.y_col = 'SalePrice'
        data = pd.read_csv(filename, index_col=['Id'])
        self.X = data[self.X_cols]
        if self.y_col in data.columns:
            self.y = np.log(data[self.y_col])
        else:
            self.y = None
            
train = HousePriceData('train.csv')
test = HousePriceData('test.csv')

test_no_nan_values(train.X), test_no_nan_values(train.y), test_no_nan_values(test.X)

(None, None, None)

Good, test passed. Now it's time to refactor. To this aim I create two python files, one for my class and one for the insanity tests. I then transformed the test to fit a unittest and rechecked if after the transformation it is going to fail for the raw data and does not fail for my transformed data. I left with 44 features from the 80+ which I considered enough to go further.

So far I tested only insanity, but a client is never going to pay for this work if the only output is that I made sure that there are no NaN values in the data. It's right time to start sanity testing as well because it helps my work staying focused. For sanity testing, I prefer using BDD frameworks for a while (e.g. [pytest-bdd]() or [behave]() ). The good thing in BDD is that it connects the testing to a human readable description of the testing steps written in Gherkin. This is useful since these are often actual requirements from the client and since it is easily readable for non-programmers as well it helps collecting external inputs to the solution. The first sanity test that I'm writing is going to test if any solution that we produce is better then the arthmetic average of the house prices. For this the Gherkin description looks like this:

```gherkin
Feature: Sanity of the model

  Scenario: Better than the simple average
    Given the trained model
    When I use the arithmetic average of the outcome as a reference 
    And the rmse of the prediction of the arithmetic average on the validation data is the reference RMSE
    And the rmse of the prediction with the trained_model on the validation data is our RMSE
    Then we see that our RMSE is better than the reference RMSE
    
```

Of course there is a proper .py file attached to this nice description. And also this fine textual description is maybe not your first iteration of the actual sanity test, maybe it is something like this:

In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def test_better_than_average(model, X, y):
    reference_score = rmse(y, np.repeat(np.mean(y), len(y)))
    y_pred = model.predict(X)
    our_score =  rmse(y, y_pred)
    assert our_score < reference_score
    
stupid_model = DummyRegressor(strategy='constant', constant=1) # so I'm using no matter what the price of the house is 1.
stupid_model = stupid_model.fit(train.X, train.y) # this model is so stupid that despite fitting it remains dumb

test_better_than_average(stupid_model, train.X, train.y)    

ValueError: could not convert string to float: 'Pave'

Hm. Expected my test fail, but not this way. Actually, this means that my data contains non numeric values. Although some models handle this, let's stick to numeric values for now. So I convert data to numeric the following way by vectorizing all text columns. But this is not so easy like calling `pd.get_dummies()` and bamm. The problem is that it could be that some categories (note, vectorizing only makes sense if we are talking about categorical variables) only present in the train or in the test data. So if you simply call `pd.get_dummies()` you get different shapes of data. 

Now, it's clear that I want to test that there are only numeric values in my data. So I added a test for that. Now to get to that you have multiple solutions:
- Use a database with schema - that's the good and proper way to it but it also takes some time to set up the schema properly for this 89 columns, so if there is another way no.
- Use some dictionary in Python and iterate over column and set up the categorical values one by one. - Nightmare to maintain such thing. Also it means you are replicating function which has been written by several people in several ways. 
- Use one of the already existing tools to get to this. - I actually opted for this solution and `pip install tdda` a nice package that check the values and validates the files. Now, the str --> categorical conversion is not built-in but the created JSON file contains the list of available values. So I added just a simple function to use that for this purpose

In [6]:
from data_analysis import HousePriceData
from pandas.api.types import is_numeric_dtype

def test_if_all_numeric(data):
    assert all(is_numeric_dtype(data.values))

constraints_filename = 'house_prices_constraints_mod.tdda'
train = HousePriceData('train.csv', constraints=constraints_filename)
test = HousePriceData('test.csv', constraints=constraints_filename)

test_if_all_numeric(train.X), test_if_all_numeric(train.y), test_if_all_numeric(test.X)

/home/jovyan/work/Documents/TDD/tdd_data_analysis/data_analysis.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X['has' + col] = (X[col] != 0).astype(int)


(None, None, None)

Now we are ready for real to fail our first sanity test. Let's see:

In [7]:
test_better_than_average(stupid_model, train.X, train.y)    

AssertionError: 

Good, failed. Now, let's pass the test. Tempting to get the XGBoost and task done, but not so fast. Although an XGBoost model sure passes the test, but so does many other model e.g. a Linear Regression. Not that fancy but a lot simpler which in this case means that is just enough to pass the test. So when you pass a test do not only think about the code that you yourself has to write but also take into account the overall complexity of your solution. So let's see if the Linear Regression passes the test

In [8]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model = model.fit(train.X, train.y)

test_better_than_average(model, train.X, train.y)

Without problems. In the refactoring step I consolidated the HousePricaData class and set up the feature file with the Gherkin program and added the necessary pytest files (I used pytest-bdd for this project). Now the next step is to have something better than Linear Regression. But, actually you don't want 'just' better, probably you want significantly better. Otherwise, how would you explain the client to but a model which predicts based on thousands of weights instand of less than a hundred? So when I'm failing my second sanity test that is going to be because I want at least less than 75% of the Linear Regression from any complex method. 

For these steps I also need a proper train and validation split, so I've splitted the data and saved the bigger part as train_short and the smaller part as validation

In [9]:
from sklearn.ensemble import ExtraTreesRegressor

def test_better_than_linear_regression(model, X_train, y_train, X_validation, y_validation, percent=75):
    lm = LinearRegression().fit(X_train, y_train)
    reference_score = rmse(y_validation, lm.predict(X_validation)) * percent /100
    y_pred = model.predict(X_validation)
    our_score =  rmse(y_validation, y_pred)
    assert our_score < reference_score
    
train = HousePriceData('train_short.csv', constraints=constraints_filename)
validation = HousePriceData('validation.csv', constraints=constraints_filename)
    
model = ExtraTreesRegressor()
model = model.fit(train.X, train.y)

test_better_than_linear_regression(model, train.X, train.y, validation.X, validation.y, percent=75)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/home/jovyan/work/Documents/TDD/tdd_data_analysis/data_analysis.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X['has' + col] = (X[col] != 0).astype(int)


AssertionError: 

Of couse it failed, actually if you look at the results you can see that the Linear Regression explains already 70% of the variance which is pretty good. After some hyperparameter tuning I found an XGBBoost model that passes this test. 

In [10]:
from xgboost import XGBRegressor

model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=400,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
       silent=True, subsample=0.7)

model = model.fit(train.X, train.y)

test_better_than_linear_regression(model, train.X, train.y, validation.X, validation.y, percent=75)

The next step was also a refactoring: I took the 'test against model' logic and organized the to the same functions to make the whole testing logic more robust.

So here I was with a working model and was able to answer already if it's better than an average prediction or a simple linear regression. Also I had a model very early on, by starting off without handling NULL values in sophisticated ways I had a model in just ~2 hour after I downloaded the data, note taking included. I would have been able to stop at basically anytime after that and had a working model and also I had a list of steps that I planned to include. I uploaded the code to github which includes the state after refactoring. In the repository you can also see that I went on adding two more sanity tests and implemented a solution to pass them. In the first test, the passing criteria was at least an RMSE that would have been enough  to land a submission in the middle of the Kaggle leaderboard (~0.13 RMSE). This was easy to fail and hard to pass :). I tried combining models, search hyperparameters, data transformations. In the end, I had to go back to the missing value problem and add the fields that had had only a small number of missing values to the data and handle the data imputation. In the next step, I tested whether I overfit the training data. Overfitting is a serious problem for a production model. Just imagine if you are overfitting on a tes set that you split from the data that you had how much you would underperform on newly collected data... 

In sum, I think it is quite clear at this point how useful to do TDD data analysis. Of course, at first you may feel that writing the test is just extra time and you of course have a model that is better than the average. I'm sure you are right, but sometimes all of us makes mistakes, tests help to reduce their effect. Also tests often live longer than the version of the code that you ended up delivering. Maybe later someone takes over but you want to make sure that his/her approach is not only different than yours but better, or at least as good. So it passes your insanity and sanity tests. Also TDD helps keeping focus. Data analysis, cleaning, imputing, modeling are all huge areas. Let's say you impute the missing values with zero, but then you think maybe you should have predicted those, then you think if you predict them then maybe you can even 'generate' extra data for training, then you start to think about how would you be able to test the effectiveness of such data generation... and you find yourself in the middle of the forest with absolutely no idea how you can get back to you original goal. TDD helps me a lot and many effective individual I know follow similar practices and what is more important I see that working in this framework makes teamwork so much more effective and fun. Try it and let me know how it works in your practice. 

In [11]:
!pytest -vv --gherkin-terminal-reporter --gherkin-terminal-reporter-expanded -p no:warnings

============================= test session starts ==============================
platform linux -- Python 3.6.3, pytest-4.3.0, py-1.8.0, pluggy-0.9.0 -- /opt/conda/bin/python
cachedir: .pytest_cache
rootdir: /home/jovyan/work/Documents/TDD/tdd_data_analysis, inifile:
plugins: bdd-3.1.0
collected 6 items                                                              

tests/insanity_test.py::DataIntegrity::test_no_nan_values PASSED         [ 16%]
tests/insanity_test.py::DataIntegrity::test_whether_only_numeric_values PASSED [ 33%]
Feature: The model is making sense
    Scenario: Better than simple average
        Given the validation data and the trained_model
        When I claim that the simplest is to take the Average of the outcome
        And I take the train_short data
        And I get the Average of the outcome from the train_short data
        And the RMSE of the prediction of the Average of the outcome for the validation as the reference score
        And the RMSE of the predict

```
<< FIN >>
```